In [1]:
%load_ext autoreload
%autoreload 2
import energyderivatives as ed

In [2]:
import matplotlib.pyplot as plt

In [3]:
import numpy as np

In [4]:
S = 100
K = 100
t = 1 / 12
sigma = 0.4
r = 0.10
b = 0.1

dt = 1 / 360

path_length = 30
mc_samples = 5000
mc_loops = 50

In [5]:
inno = ed.monte_carlo_options.NormalSobolInnovations
path = ed.monte_carlo_options.WienerPath
payoff = ed.monte_carlo_options.PlainVanillaPayoff

In [6]:
mc = ed.monte_carlo_options.MonteCarloOption(
    mc_loops,
    path_length,
    mc_samples,
    dt,
    S,
    K,
    t,
    r,
    b,
    sigma,
    inno,
    path,
    payoff,
    # eps=eps,
    trace=False,
    antithetic=False,
)


In [7]:
ee = mc.call()
ee

/home/bcho/miniconda3/envs/main/lib/python3.9/site-packages/scipy/stats/_qmc.py:1078: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  warnings.warn("The balance properties of Sobol' points require"


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
max(ee[2])

In [ ]:
ee[1].generate_path()

In [ ]:
tt = (b - (sigma ** 2) / 2) * dt + sigma * np.sqrt(dt)  * ee[0]

In [ ]:
max(tt)

In [ ]:
np.mean(ee[1].generate_path())

In [ ]:
ee[1].generate_path()

In [ ]:
plt.hist(ee[1].generate_path())